In [1]:
!pip install mrjob
from IPython.display import Image

In [1]:
#!mkdir HW4

In [12]:
!pwd

/Users/z00193k/Dropbox/DataScienceProgram/Day4


In [2]:
!find / -name "hadoop*streaming*" -print 1> tmp.hadoopJar 2> /dev/null

In [3]:
!cat tmp.hadoopJar

/Users/z00193k/.Trash/hadoop 3.56.32 PM/2.7.2/libexec/share/doc/hadoop/hadoop-streaming
/Users/z00193k/.Trash/hadoop 3.56.32 PM/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar
/Users/z00193k/.Trash/hadoop 3.56.32 PM/2.7.2/libexec/share/hadoop/tools/sources/hadoop-streaming-2.7.2-sources.jar
/Users/z00193k/.Trash/hadoop 3.56.32 PM/2.7.2/libexec/share/hadoop/tools/sources/hadoop-streaming-2.7.2-test-sources.jar
/usr/local/Cellar/hadoop/2.7.2/libexec/share/doc/hadoop/hadoop-streaming
/usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar
/usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/sources/hadoop-streaming-2.7.2-sources.jar
/usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/sources/hadoop-streaming-2.7.2-test-sources.jar


### 3.HW4

Back to Table of Contents

ASSIGNMENTS using Mrjob or Hadoop Streaming and Python (classification using Multinomial Naive Bayes)
#### * Dont forget to create a new subdirectory for this homework (on your local machine/virtual machine, and on HDFS)

### 3.HW4.0.1.

In Hadoop streaming write code to determing the top-10 most frequent using words present in the text version of Alice’s Adventures in Wonderland. (You can obtain a free plain text version of the book, along with many others, from here. 

##### First use one reducer and report your result.

    Please leverage the following notebook for this HW problem: Total Sort Notebook

For your result please create a text file named alice_sorted_words.txt containing an listing of all the words, and the number of times each occurs, in the text version of Alice’s Adventures in Wonderland in the following sort order: WordCount descreasing, and where words that have the same count, they be ordered in alphabetical increasing. (You can obtain a free plain text version of the book, along with many others, from here The first 10 lines of your output file should look something like this:


In [3]:
# Scraping the book from the website

!curl 'http://www.gutenberg.org/cache/epub/11/pg11.txt' -o alicesTExtFilename.txt
!head alicesTExtFilename.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  163k  100  163k    0     0  46663      0  0:00:03  0:00:03 --:--:-- 46662
﻿Project Gutenberg's Alice's Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Alice's Adventures in Wonderland



# Using Hadoop Streaming

In [4]:
%%writefile HW4/wcmapper1.py
#!/usr/bin/env python

import sys

for line in sys.stdin:
    for word in line.split():
        print '%s\t%s' % (word.lower(), 1)

Overwriting HW4/wcmapper1.py


In [5]:
!chmod a+x HW4/wcmapper1.py

In [6]:
!echo "you are my you world bro Really bro" | HW4/wcmapper1.py

you	1
are	1
my	1
you	1
world	1
bro	1
really	1
bro	1


In [7]:
%%writefile HW4/wcreducer1.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting HW4/wcreducer1.py


In [8]:
!chmod a+x HW4/wcreducer1.py

In [12]:
!cat alicesTExtFilename.txt | HW4/wcmapper1.py | sort -k1,1 | HW4/wcreducer1.py | sort -k2,2nr -k1,1

reporter:counter:Reducer Counters,Calls,1
the	1777
and	833
to	782
a	670
of	610
she	518
said	421
in	412
it	374
was	334
you	332
i	260
as	258
that	237
alice	221
at	220
with	217
her	208
had	176
all	173
for	161
be	157
on	152
this	150
or	143
very	140
not	129
so	127
'i	121
but	121
little	120
they	120
he	113
out	98
is	97
his	95
about	92
what	92
if	91
up	83
have	82
were	82
one	81
project	81
went	79
down	78
no	77
alice,	76
by	76
would	76
like	75
when	74
any	71
your	69
do	68
into	67
could	66
there	66
its	63
thought	63
*	60
an	60
are	59
then	59
who	57
mock	56
my	56
'and	55
quite	55
alice.	54
gutenberg-tm	54
don't	53
must	51
see	51
some	51
their	51
did	50
how	50
them	49
just	48
began	47
time	47
which	47
from	46
know	46
me	46
only	46
such	46
got	45
looked	45
get	43
can	41
it,	41
much	41
after	40
herself	40
more	40
never	40
off	40
other	40
work	40
'you	39
go	39
great	39
way	39
'but	38
came	38
think	38
i'm	37
will	37
been	36
queen	35
say	35
thing	35
'what	34
turtle	33
large	32
without	32
again,	31
fou

In [14]:
!hdfs dfs -ls

16/09/04 18:16:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   1 z00193k supergroup     167518 2016-09-04 17:46 alicesTExtFilename.txt
drwxr-xr-x   - z00193k supergroup          0 2016-09-04 17:46 wordcount-output


In [18]:
!hdfs dfs -rm alicesTExtFilename.txt 
!hdfs dfs -copyFromLocal alicesTExtFilename.txt 
!hdfs dfs -rm -r wordcount-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
#dataDir="/home/cloudera/Dropbox/DataScienceProgram/Day4/HW4/"

!hadoop jar /usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
   -mapper /Users/z00193k/Dropbox/DataScienceProgram/Day4/HW4/wcmapper1.py \
   -reducer /Users/z00193k/Dropbox/DataScienceProgram/Day4/HW4/wcreducer1.py \
   -combiner /Users/z00193k/Dropbox/DataScienceProgram/Day4/HW4/wcreducer1.py \
   -input alicesTExtFilename.txt \
   -output wordcount-output  \
   -numReduceTasks 1
   #--D mapreduce.job.reduces=2  depecated

16/09/04 18:25:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 18:25:10 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted alicesTExtFilename.txt
16/09/04 18:25:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 18:25:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 18:25:12 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output
16/09/04 18:25:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 18:25:14 INFO Configuration.deprecation: session.id is deprecated. In

In [9]:
!hdfs dfs -cat wordcount-output/part-0000* 

16/09/04 20:15:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"'tis	1
"--said	1
"come	2
"coming	1
"defects,"	1
"edwin	1
"french,	1
"he's	1
"how	2
"i	7
"i'll	2
"information	1
"it"	2
"keep	1
"let	1
"much	1
"plain	2
"poison"	1
"project	5
"purpose"?'	1
"right	1
"such	1
"there	2
"there's	1
"they	1
"too	1
"turtle	1
"twinkle,	1
"uglification,"'	1
"up	1
"what	2
"who	1
"william	1
"with	1
"you	3
#11]	1
$5,000)	1
'"--found	1
'"miss	1
'"we	1
'"what	1
'"will	1
''tis	2
'--and	2
'--as	1
'--but	1
'--change	1
'--for	1
'--i	1
'--it	1
'--likely	1
'--mystery,	1
'--or	1
'--so	2
'--that	1
'--well	1
'--where's	1
'--yes,	1
'--you	1
'a	11
'advance	1
'after	2
'ah!	2
'ah,	2
'ahem!'	1
'alice!'	1
'all	3
'allow	1
'an	1
'and	55
'any	1
'anything	1
'are	7
'arrum.')	1
'as	7
'as-is'	1
'at	3
'back	1
'beautiful	2
'because	3
'begin	1
'behead	1
'besides,	1
'boots	1
'bring	1
'but	38
'but,	1
'by-the-bye,	1
'call	4
'can	1
'can't	1
'certainly	

In [14]:
!hdfs dfs -cat wordcount-output/part-0000* > output1.txt

16/09/04 20:23:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
!hdfs dfs -ls wordcount-output

16/09/04 20:23:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   1 z00193k supergroup          0 2016-09-04 18:25 wordcount-output/_SUCCESS
-rw-r--r--   1 z00193k supergroup      55096 2016-09-04 18:25 wordcount-output/part-00000


In [20]:
!python --version

Python 2.7.12 :: Anaconda 2.5.0 (x86_64)


### Creating identity mapper for job2. Top 10 words sorted first on count and then on alphabet

In [17]:
!hdfs dfs -rm output1.txt
!hdfs dfs -put output1.txt   
!hdfs dfs -rm -r output
!hadoop jar /usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=2 \
-D stream.map.output.field.separator="\t" \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-mapper /bin/cat \
-reducer /bin/cat \
-input output1.txt \
-output output \

16/09/04 22:54:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 22:54:55 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted output1.txt
16/09/04 22:54:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 22:54:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 22:54:57 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted output
16/09/04 22:54:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/09/04 22:54:59 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metric

In [20]:
!hdfs dfs -cat output/part-0000* | head -n20

16/09/04 22:58:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
the	1777	
and	833	
to	782	
a	670	
of	610	
she	518	
said	421	
in	412	
it	374	
was	334	
you	332	
i	260	
as	258	
that	237	
alice	221	
at	220	
with	217	
her	208	
had	176	
all	173	
cat: Unable to write to output stream.


In [19]:
!hdfs dfs -cat output/part-0000* | head -n20 > output_top20.txt

16/09/04 22:57:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
cat: Unable to write to output stream.


### ********* End of Question 4.0.1 *********

# HW4.0.2. (Optional) Total Sort using mulitple reducers

Using MrJob determine the top-10 occurring tokens (most frequent tokens) using three (3) reducers. HINT: Check out the following notebook for examples of total sort using multiple reducers in MRJob: Total Sort Notebook for Hadoop Streaming and for MRJob


In [1]:
%%writefile generate_numbers.py
#!/usr/bin/python
words = ["cell","center","change","cluster","clustering","code","computational","compute","computing","consists",\
         "contour","corresponding","creating","current","dataset","def","descent","descent","develop","different",\
         "distributed","do","document","done","driver","drivers","during","efficient","evaluate","experiements"]
import random

N = 30
for n in range(N):
    print random.randint(0,N),"\t",words[n]

Overwriting generate_numbers.py


In [2]:
!chmod +x generate_numbers.py;
!./generate_numbers.py > generate_numbers.output
!wc -l generate_numbers.output

      30 generate_numbers.output


In [3]:
%%writefile shuffle.py
import sys
import random

if __name__ == '__main__':
    with open(sys.argv[1], 'r') as f:
        flist = f.readlines()
        random.shuffle(flist)

    for line in flist:
        print line.strip()

Overwriting shuffle.py


In [24]:
!python shuffle.py generate_numbers.output > generate_numbers.output.randomized
!cat generate_numbers.output.randomized

16 	contour
23 	do
30 	dataset
29 	def
10 	computational
2 	document
28 	clustering
20 	driver
11 	change
0 	evaluate
12 	cluster
27 	descent
6 	done
0 	computing
30 	drivers
27 	develop
24 	code
9 	corresponding
23 	efficient
14 	center
12 	creating
20 	compute
19 	cell
2 	consists
1 	current
5 	descent
12 	during
16 	different
2 	experiements
23 	distributed


### III.D.4. MRJob Multiple Reducers - With Ordered Partitons

In [8]:
%%writefile MRJob_multipleReducerTotalOrderSort.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol
from operator import itemgetter
import numpy as np

class MRJob_multipleReducerTotalOrderSort(MRJob):
    
    # The following three settings are your sorting best friends:
    MRJob.SORT_VALUES = True 
    
    # The protocols are critical. It will not work without these:
    INTERNAL_PROTOCOL = RawProtocol
    OUTPUT_PROTOCOL = RawProtocol
 
    def __init__(self, *args, **kwargs):
        super(MRJob_multipleReducerTotalOrderSort, self).__init__(*args, **kwargs)
        self.N = 30
        self.NUM_REDUCERS = 1
    
    def mapper_partitioner_init(self):
        
        def makeKeyHash(key, num_reducers):
            byteof = lambda char: int(format(ord(char), 'b'), 2)
            current_hash = 0
            for c in key:
                current_hash = (current_hash * 31 + byteof(c))
            return current_hash % num_reducers
        
        # printable ascii characters, starting with 'A'
        keys = [str(unichr(i)) for i in range(65,65+self.NUM_REDUCERS)]
        partitions = []
        
        for key in keys:
            partitions.append([key, makeKeyHash(key, self.NUM_REDUCERS)])

        parts = sorted(partitions,key=itemgetter(1))
        self.partition_keys = list(np.array(parts)[:,0])
        
        self.partition_file = np.arange(0,self.N,self.N/(self.NUM_REDUCERS))[::-1]
        
    def mapper_partition(self, _, line):
        line = line.strip()
        key,value = line.split('\t')
        
        # Prepend the approriate key by finding the bucket, and using the index to fetch the key.
        for idx in xrange(self.NUM_REDUCERS):
            if float(key) > self.partition_file[idx]:
                yield str(self.partition_keys[idx]),key+"\t"+value
                break
        
        
            
    def reducer(self,key,value):
        for v in value:
            yield key,v
            # To omit the partition key, specify 'None'
            # We are keeping it for illutration purposes
    
    def steps(self):
        
        JOBCONF_STEP1 = {
            'stream.num.map.output.key.field':3,
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'stream.map.output.field.separator':"\t",
            'mapreduce.partition.keypartitioner.options':'-k1,1',
            'mapreduce.partition.keycomparator.options':'-k2,2nr -k3,3',
            'mapred.reduce.tasks': self.NUM_REDUCERS,
            'partitioner':'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
        }
        return [MRStep(jobconf=JOBCONF_STEP1,
                    mapper_init=self.mapper_partitioner_init,
                    mapper=self.mapper_partition,
                    reducer=self.reducer)
                ]


if __name__ == '__main__':
    MRJob_multipleReducerTotalOrderSort.run()

Overwriting MRJob_multipleReducerTotalOrderSort.py


In [9]:
!hdfs dfs -rm -r total_order_sort
!python MRJob_multipleReducerTotalOrderSort.py generate_numbers.output \

16/09/05 09:06:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `total_order_sort': No such file or directory
No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/MRJob_multipleReducerTotalOrderSort.z00193k.20160905.033617.428896
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/MRJob_multipleReducerTotalOrderSort.z00193k.20160905.033617.428896/output...
A	1 	drivers
A	10 	computational
A	11 	current
A	11 	different
A	11 	done
A	11 	during
A	12 	cell
A	13 	creating
A	15 	consists
A	15 	def
A	15 	document
A	2 	computing
A	2 	efficient
A	2 	experiements
A	20 	cluster
A	21 	change
A	22 	code
A	22 	driver
A	23 	evaluate
A	24 	dataset
A	24 	descent
A	26 	contour
A	27 	dist

In [10]:
print "="*100
print "Total Order Sort with multiple reducers - notice that the part files are also in order."
print "="*100
print "/part-00000"
print "-"*100
!hdfs dfs -cat /user/koza/total_order_sort/part-00000
print "-"*100
print "/part-00001"
print "-"*100
!hdfs dfs -cat /user/koza/total_order_sort/part-00001
print "-"*100
print "/part-00002"
print "-"*100
!hdfs dfs -cat /user/koza/total_order_sort/part-00002

Total Order Sort with multiple reducers - notice that the part files are also in order.
/part-00000
----------------------------------------------------------------------------------------------------
16/09/05 09:09:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
cat: `/user/koza/total_order_sort/part-00000': No such file or directory
----------------------------------------------------------------------------------------------------
/part-00001
----------------------------------------------------------------------------------------------------
16/09/05 09:09:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
cat: `/user/koza/total_order_sort/part-00001': No such file or directory
----------------------------------------------------------------------------------------------------
/part-00002
------------------------------


## HW4.0.3.

Define big data. Provide an example of a big data problem in your domain of expertise.

What is a race condition in the context of parallel computation? Give an example. What is MapReduce? How does it differ from Hadoop?


### Ans. 




### HW4.0.4:

Here is an example of functional programming in basic python in terms of mappers and reducers (by way of example):
#### EXAMPLE Mapper functions in Python

def fahrenheit(T): return ((float(9)/5)*T + 32)

def celsius(T): return (float(5)/9)*(T-32)

temperatures = (36.5, 37, 37.5, 38, 39) F = map(fahrenheit, temperatures)
#### returns 97.7 98.6 99.5 100.4 102.2

C = map(celsius, F)
#### EXAMPLE Reducer function in Python

import functools functools.reduce(lambda x,y: x+y, [47,11,42,13])
#### returns 113

import functools as reduce print "Average temp is %fF" % (reduce(lambda x,y: x+y, F)/len(F) )
#### returns Average temp is 99.68F

Which programming paradigm is Hadoop based on? Explain and give a simple example of functional programming in raw python code and show the code running. E.g., in raw python find the average length of a string in and of strings using a python "map-reduce" (functional programming) job (similar in style to the above). Alternatively, you can do this in python Hadoop Streaming.

strings = ["str1", "string2", "w261", "MAchine learning at SCALE"] .......

import functools as reduce temperatures = (36.5, 37, 37.5, 38, 39) F = map(fahrenheit, temperatures) print "Average temp is %fF" % (reduce(lambda x,y: x+y, F)/len(F) )
#### returns Average temp is 99.68F

map(sqr, items)

#

# 3. HW4.2 WordCount of SPAM data

### HW4.2: 
WORDCOUNT Using the Enron dataset and MRJob, write the mapper/reducer job that will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.
CROSSCHECK your result with the following Unix commands: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l 8 #NOTE "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

In [24]:
%%writefile wc_mrjob.py

import mrjob
from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"[\w']+")


class MRWordFreqCount(MRJob):

    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner(self, word, counts):
        yield (word, sum(counts))

    def reducer(self, word, counts):
        yield (word, sum(counts))


if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting wc_mrjob.py


In [25]:
!python wc_mrjob.py HW4/enronemail_1h.txt \

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/wc_mrjob.z00193k.20160905.054225.976962
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/wc_mrjob.z00193k.20160905.054225.976962/output...
"'"	3
"'bcli"	1
"'benewm"	1
"'bjeffrie"	1
"'blong"	1
"'bredd"	1
"'celias"	1
"'cenochs"	1
"'chuck"	1
"'colliw"	1
"'connie"	1
"'ctise"	1
"'davidyi"	1
"'dfelsinger"	1
"'dkohler"	1
"'dmao"	1
"'don"	1
"'doug"	1
"'dronn"	1
"'five"	1
"'gary"	1
"'glen"	1
"'hal"	1
"'hans"	1
"'harry_wijsman"	1
"'hoferc"	1
"'iccenergy"	1
"'ilydiatt"	1
"'jeff"	1
"'jlopes"	1
"'jnexon"	1
"'john"	1
"'jones"	1
"'jonesg"	1
"'jshorter"	1
"'kelly"	1
"'kenny"	1
"'larry"	1
"'llittle"	1
"'mcclankg"	1
"'mcosta"	1
"'michael_huse"	1
"'mmce"	1
"'mperkins"	1
"'msessa"	1
"'njwa"	1
"'pete"	1
"'prez"	1
"'priscilla"	1
"'pwarden"	1
"'randall"	1
"'randy"	1
"'reason"	1
"'repling"	1
"'richard"	1
"'richarddaniel"	1
"'rsbaker"	1

### HW4.2.1 Total Sort using a single reducer¶
Using MrJob determine the top-10 occurring tokens (most frequent tokens) using a single reducer.

In [15]:
%%writefile wc_mrjobtop.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs) 


if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting wc_mrjobtop.py


In [16]:
!python wc_mrjobtop.py HW4/enronemail_1h.txt \

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/wc_mrjobtop.z00193k.20160905.131614.070813
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/wc_mrjobtop.z00193k.20160905.131614.070813/output...
1247	"the"
Removing temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/wc_mrjobtop.z00193k.20160905.131614.070813...


## 3. HW4.3 Multinomial NAIVE BAYES with NO Smoothing using a single reducer

Using the Enron data from HW1 and Hadoop MapReduce (or MRJob), write a mapper/reducer job(s) that will both learn Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM

E.g., “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. Multiplying lots of probabilities, which are between 0 and 1, can result in floating-point underflow. Since log(xy) = log(x) + log(y), it is better to perform all computations by summing logs of probabilities rather than multiplying probabilities. Please pay attention to probabilites that are zero! They will need special attention. Count up how many times you need to process a zero probabilty for each class and report.

##### Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes Classifier. Plot a histogram of the posterior probabilities (i.e., Pr(Class|Doc)) for each class over the training set. Summarize what you see.

Error Rate = misclassification rate with respect to a provided set (say training set in this case. This is not a good practice in real time!). It is more formally defined here:

Let DF represent the evalution set in the following: Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}| / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

NOTE: please assume one reducer.

In [ ]:
import sys, re

def cleaner(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    words = text.split()
    for word in words :
        if word[0]>="a" and word[0]<="z":
            cleaned.append(word)
        else:
            pass
    return cleaned


In [75]:
%%writefile SpamTrainer.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

#Step 1. Getting the word and wordcount for the the documents

def cleaner(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    words = text.split()
    for word in words :
        if word[0]>="a" and word[0]<="z":
            cleaned.append(word)
        else:
            pass
    return cleaned


class SpamTrainer(MRJob):
    #Writing the reducer for counting the word and count
    def __init__(self, *args, **kwargs):
        super(SpamTrainer, self).__init__(*args, **kwargs)
        self.modelStats = {}

    def jobconf(self):
        orig_jobconf = super(SpamTrainer, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    
    def mapper(self, _, line):
        docID, docClass, text = line.split("\t", 2)
        words=cleaner(text)
        if docClass == "1":
            yield("Priori", "1, 0")
            for word in words:
                yield (word, "1, 0")
        else:
            yield("Priori", "0, 1")
            for word in words:
                yield (word, "0, 1")
                
    def reducer(self, word, values):
        w0Total=0
        w1Total=0
        for value in values:
            w0, w1 = value.split(",")
            w0Total+=float(w0)
            w1Total+=float(w1)
        self.modelStats[word]= [w0Total, w1Total]
        word, [w0Total, w1Total] 
        
    def reducer_final(self):
        class0Total = 0
        class1Total = 0
        for k in self.modelStats.keys():
            if k != "Priori":
                class0Total += self.modelStats[k][0]
                class1Total += self.modelStats[k][1]
        vocabularySize = len(self.modelStats.keys()) -1  #ignore TomsPriors
        #some yields to see some model internal parameters
        #yield ("defaultPrior 0 class", class0Total+vocabularySize)
        #yield ("defaultPrior 1 class", class1Total+vocabularySize)
        #yield ("count 0 class", class0Total)
        #yield ("count 1 class", class1Total)
        #yield ("vocabularySize", vocabularySize)
        
        #calculate priors 
        classCount0, classCount1 = self.modelStats.get("Priori")
        del self.modelStats["Priori"]
        total = classCount0 + classCount1
        yield("Priori", ','.join(str(j) for j in [classCount0, classCount1, classCount0/total, classCount1/total])) 
        for k in self.modelStats.keys():
            yield(k, ','.join(str(j) for j in [self.modelStats[k][0],
                      self.modelStats[k][1],
                      (self.modelStats[k][0]) /(class0Total), 
                      (self.modelStats[k][1])/(class1Total)]))  
            
if __name__ == '__main__':
    SpamTrainer.run()

Overwriting SpamTrainer.py


In [76]:
!python SpamTrainer.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainer.z00193k.20160906.020538.079871
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainer.z00193k.20160906.020538.079871/output...
"Priori"	"44.0,56.0,0.44,0.56"
"stock"	"2.0,3.0,0.000110411836149,0.00022453409176"
"limited"	"10.0,4.0,0.000552059180744,0.000299378789013"
"entergyr"	"0.0,1.0,0.0,7.48446972532e-05"
"additionally"	"1.0,1.0,5.52059180744e-05,7.48446972532e-05"
"doubts"	"1.0,0.0,5.52059180744e-05,0.0"
"dynamic"	"0.0,2.0,0.0,0.000149689394506"
"externally"	"2.0,0.0,0.000110411836149,0.0"
"yellow"	"1.0,0.0,5.52059180744e-05,0.0"
"barraged"	"1.0,0.0,5.52059180744e-05,0.0"
"prefix"	"1.0,0.0,5.52059180744e-05,0.0"
"sleep"	"2.0,0.0,0.000110411836149,0.0"
"invovled"	"0.0,2.0,0.0,0.000149689394506"
"railing"	"1.0,0.0,5.52059180744e-05,0.0"
"appetite"	"2.0,0.0,0.000110411836149,0.0"
"hate"	"4.0,

In [77]:
!python SpamTrainer.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt > HW4/SpamTrainer.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainer.z00193k.20160906.020541.018635
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainer.z00193k.20160906.020541.018635/output...
Removing temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainer.z00193k.20160906.020541.018635...


In [78]:
!cat HW4/SpamTrainer.txt

"Priori"	"44.0,56.0,0.44,0.56"
"stock"	"2.0,3.0,0.000110411836149,0.00022453409176"
"limited"	"10.0,4.0,0.000552059180744,0.000299378789013"
"entergyr"	"0.0,1.0,0.0,7.48446972532e-05"
"additionally"	"1.0,1.0,5.52059180744e-05,7.48446972532e-05"
"doubts"	"1.0,0.0,5.52059180744e-05,0.0"
"dynamic"	"0.0,2.0,0.0,0.000149689394506"
"externally"	"2.0,0.0,0.000110411836149,0.0"
"yellow"	"1.0,0.0,5.52059180744e-05,0.0"
"barraged"	"1.0,0.0,5.52059180744e-05,0.0"
"prefix"	"1.0,0.0,5.52059180744e-05,0.0"
"sleep"	"2.0,0.0,0.000110411836149,0.0"
"invovled"	"0.0,2.0,0.0,0.000149689394506"
"railing"	"1.0,0.0,5.52059180744e-05,0.0"
"appetite"	"2.0,0.0,0.000110411836149,0.0"
"hate"	"4.0,0.0,0.000220823672298,0.0"
"worked"	"2.0,1.0,0.000110411836149,7.48446972532e-05"
"forget"	"1.0,0.0,5.52059180744e-05,0.0"
"looking"	"4.0,0.0,0.000220823672298,0.0"
"fronts"	"0.0,1.0,0.0,7.48446972532e-05"
"founder"	"1.0,0.0,5.52059180744e-05,0.0"
"granting"	"1.0,0.0,5.52059180744e-05,0.0"
"electricity"	"0.0,1.0,0.0,7.48

In [46]:
# load model file as strings (it is not tokenized; that will come next)
modelStats = {}
recordStrs = [s.split('\n')[0].split('\t') for s in open("HW4/SpamTrainer.txt").readlines()]
for word, statsStr in recordStrs:
    print word, "-->", statsStr

"Priori" --> "44.0,56.0,0.44,0.56"
"stock" --> "2.0,3.0,0.000129427499029,0.000217084554434"
"limited" --> "10.0,4.0,0.000474567496441,0.000271355693042"
"entergyr" --> "0.0,1.0,4.31424996764e-05,0.000108542277217"
"additionally" --> "1.0,1.0,8.62849993529e-05,0.000108542277217"
"doubts" --> "1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"dynamic" --> "0.0,2.0,4.31424996764e-05,0.000162813415825"
"externally" --> "2.0,0.0,0.000129427499029,5.42711386085e-05"
"yellow" --> "1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"barraged" --> "1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"prefix" --> "1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"sleep" --> "2.0,0.0,0.000129427499029,5.42711386085e-05"
"invovled" --> "0.0,2.0,4.31424996764e-05,0.000162813415825"
"railing" --> "1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"appetite" --> "2.0,0.0,0.000129427499029,5.42711386085e-05"
"hate" --> "4.0,0.0,0.000215712498382,5.42711386085e-05"
"worked" --> "2.0,1.0,0.000129427499029,0.000108542277217"
"fo

In [50]:
# load model file
# notice the string quotes around the frequncy and probabilits
# we need to fix that next 
modelStats = {}
recordStrs = [s.split('\n')[0].split('\t') for s in open("HW4/SpamTrainer.txt").readlines()]
for word, statsStr in recordStrs:
    modelStats[word] =  statsStr.split(",")
modelStats

{'"request"': ['"11.0', '3.0', '0.000517709996117', '0.000217084554434"'],
 '"harrison"': ['"0.0', '2.0', '4.31424996764e-05', '0.000162813415825"'],
 '"francisco"': ['"0.0', '1.0', '4.31424996764e-05', '0.000108542277217"'],
 '"sds"': ['"1.0', '0.0', '8.62849993529e-05', '5.42711386085e-05"'],
 '"heard"': ['"2.0', '0.0', '0.000129427499029', '5.42711386085e-05"'],
 '"content"': ['"12.0', '1.0', '0.000560852495794', '0.000108542277217"'],
 '"factual"': ['"0.0', '1.0', '4.31424996764e-05', '0.000108542277217"'],
 '"placed"': ['"1.0', '0.0', '8.62849993529e-05', '5.42711386085e-05"'],
 '"communicate"': ['"0.0', '2.0', '4.31424996764e-05', '0.000162813415825"'],
 '"done"': ['"0.0', '6.0', '4.31424996764e-05', '0.000379897970259"'],
 '"present"': ['"0.0', '2.0', '4.31424996764e-05', '0.000162813415825"'],
 '"deposit"': ['"4.0', '0.0', '0.000215712498382', '5.42711386085e-05"'],
 '"involvement"': ['"1.0', '0.0', '8.62849993529e-05', '5.42711386085e-05"'],
 '"competing"': ['"2.0', '1.0', '0.

In [87]:
%%writefile SpamClassifier.py
#HW 1.3
 
from mrjob.job import MRJob
import sys, re, string, operator, math, os

def cleaner(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    words = text.split()
    for word in words :
        if word[0]>="a" and word[0]<="z":
            cleaned.append(word)
        else:
            pass
    return cleaned

class SpamClassifier(MRJob):

    def __init__(self, *args, **kwargs):
        super(SpamClassifier, self).__init__(*args, **kwargs)
        self.nb_wordstats={}
        self.classification_output ={}
        
    def jobconf(self):
        orig_jobconf = super(SpamClassifier, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf    

    #load model, that is, word proportional table from file output of trainer
    def mapper_init(self):
        self.nb_wordstats = {}
        #Create a dictionary of key (word), value(list of length 4) pairs
        recordStrs = [s.split('\n')[0].split('\t') for s in open("NB_Model.txt").readlines()]
        for word, statsStr in recordStrs:
            self.nb_wordstats[word] = map(float, statsStr.split(","))
            #Get prior probabilities
            self.prior_Spam = math.log(self.nb_wordstats["PriorsNB"][0])
            self.prior_notSpam = math.log(self.nb_wordstats["PriorsNB"][1])
        
    def mapper(self,_,line):
        docID,docClass,text=line.split('\t',2)
        words = cleaner(text)
        for word in words:
            try:
                P_spam_word = math.log(self.nb_wordstats[word][0]) if self.nb_wordstats[word][0]>0.0 else math.log(1)
            except:
                P_spam_word = float(0)
            try:
                P_notspam_word = math.log(self.nb_wordstats[word][1]) if self.nb_wordstats[word][1]>0.0 else math.log(1)
            except:
                P_notspam_word = float(0)
        P_spam_givenwords = P_spam_word + self.prior_Spam        
        P_notspam_givenwords = P_notspam_word + self.prior_notSpam
        class_predicted = 1 if P_spam_givenwords > P_notspam_givenwords else 0
        yield('Total_Docs', '0,1')
        if int(docClass) != class_predicted: yield ('Misclassification', '0,1')  
        yield(docID, ','.join(str(j) for j in [docClass, class_predicted]))
    
    def reducer(self,docID,values):
        Total_counter =  0
        for value in values:
            correct,info = value.split(',')
            Total_counter +=int(info)
        self.classification_output[docID] = Total_counter

    def reducer_final(self):
        print 'Total number of docs is ', self.classification_output.get("Total_Docs")
        print 'Error rate is ', float(self.classification_output.get("Misclassification"))/float(self.classification_output.get("Total_Docs"))

if __name__ == '__main__':
    SpamClassifier.run()

Overwriting SpamClassifier.py


In [88]:
!python SpamClassifier.py --jobconf mapred.reduce.tasks=1 HW4/SpamTrainer.txt

  File "SpamClassifier.py", line 24
SyntaxError: Non-ASCII character '\xe2' in file SpamClassifier.py on line 24, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details


# 3. HW4.4 Multinomial Naive Bayes with Smoothing

In [89]:
%%writefile SpamTrainerWithSmoothing.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

#Step 1. Getting the word and wordcount for the the documents

def cleaner(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    words = text.split()
    for word in words :
        if word[0]>="a" and word[0]<="z":
            cleaned.append(word)
        else:
            pass
    return cleaned


class SpamTrainerWithSmoothing(MRJob):
    #Writing the reducer for counting the word and count
    def __init__(self, *args, **kwargs):
        super(SpamTrainerWithSmoothing, self).__init__(*args, **kwargs)
        self.modelStats = {}

    def jobconf(self):
        orig_jobconf = super(SpamTrainerWithSmoothing, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    
    def mapper(self, _, line):
        docID, docClass, text = line.split("\t", 2)
        words=cleaner(text)
        if docClass == "1":
            yield("Priori", "1, 0")
            for word in words:
                yield (word, "1, 0")
        else:
            yield("Priori", "0, 1")
            for word in words:
                yield (word, "0, 1")
                
    def reducer(self, word, values):
        w0Total=0
        w1Total=0
        for value in values:
            w0, w1 = value.split(",")
            w0Total+=float(w0)
            w1Total+=float(w1)
        self.modelStats[word]= [w0Total, w1Total]
        word, [w0Total, w1Total] 
        
    def reducer_final(self):
        class0Total = 0
        class1Total = 0
        for k in self.modelStats.keys():
            if k != "Priori":
                class0Total += self.modelStats[k][0]
                class1Total += self.modelStats[k][1]
        vocabularySize = len(self.modelStats.keys()) -1  #ignore TomsPriors
        #some yields to see some model internal parameters
        #yield ("defaultPrior 0 class", class0Total+vocabularySize)
        #yield ("defaultPrior 1 class", class1Total+vocabularySize)
        #yield ("count 0 class", class0Total)
        #yield ("count 1 class", class1Total)
        #yield ("vocabularySize", vocabularySize)
        
        #calculate priors 
        classCount0, classCount1 = self.modelStats.get("Priori")
        del self.modelStats["Priori"]
        total = classCount0 + classCount1
        yield("Priori", ','.join(str(j) for j in [classCount0, classCount1, classCount0/total, classCount1/total])) 
        for k in self.modelStats.keys():
            yield(k, ','.join(str(j) for j in [self.modelStats[k][0],
                      self.modelStats[k][1],
                      (self.modelStats[k][0] + 1) /(class0Total + vocabularySize), 
                      (self.modelStats[k][1] +1)/(class1Total+vocabularySize)]))  

if __name__ == '__main__':
    SpamTrainerWithSmoothing.run()

Overwriting SpamTrainerWithSmoothing.py


In [90]:
!python SpamTrainerWithSmoothing.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing.z00193k.20160906.021443.491609
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing.z00193k.20160906.021443.491609/output...
"Priori"	"44.0,56.0,0.44,0.56"
"stock"	"2.0,3.0,0.000129427499029,0.000217084554434"
"limited"	"10.0,4.0,0.000474567496441,0.000271355693042"
"entergyr"	"0.0,1.0,4.31424996764e-05,0.000108542277217"
"additionally"	"1.0,1.0,8.62849993529e-05,0.000108542277217"
"doubts"	"1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"dynamic"	"0.0,2.0,4.31424996764e-05,0.000162813415825"
"externally"	"2.0,0.0,0.000129427499029,5.42711386085e-05"
"yellow"	"1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"barraged"	"1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"prefix"	"1.0,0.0,8.62849993529e-05,5.42711386085e-05"
"sleep"	"2.0,0.0,0.000129427499029,5.42711386085e-05"
"

In [91]:
!python SpamTrainerWithSmoothing.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt >HW4/SpamClassifierWithSmoothing.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing.z00193k.20160906.021615.933399
Running step 1 of 1...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing.z00193k.20160906.021615.933399/output...
Removing temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing.z00193k.20160906.021615.933399...


In [92]:
!python SpamClassifier.py --jobconf mapred.reduce.tasks=1 HW4/SpamClassifierWithSmoothing.txt

  File "SpamClassifier.py", line 24
SyntaxError: Non-ASCII character '\xe2' in file SpamClassifier.py on line 24, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details


## 3. HW4.5 Remove words with frequency of less than three (3) in the training set

In [73]:
%%writefile SpamTrainerWithSmoothing_gt3.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

#Step 1. Getting the word and wordcount for the the documents

def cleaner(text):
    cleaned =[]
    text = text.strip()
    text = re.sub( '\W+', ' ', text.lower() )
    words = text.split()
    for word in words :
        if word[0]>="a" and word[0]<="z":
            cleaned.append(word)
        else:
            pass
    return cleaned


class SpamTrainerWithSmoothing_gt3(MRJob):
    #Writing the reducer for counting the word and count
    def __init__(self, *args, **kwargs):
        super(SpamTrainerWithSmoothing_gt3, self).__init__(*args, **kwargs)
        self.modelStats = {}

    def jobconf(self):
        orig_jobconf = super(SpamTrainerWithSmoothing_gt3, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    
    def mapper(self, _, line):
        docID, docClass, text = line.split("\t", 2)
        words=cleaner(text)
        if docClass == "1":
            yield("Priori", "1, 0")
            for word in words:
                yield (word, "1, 0")
        else:
            yield("Priori", "0, 1")
            for word in words:
                yield (word, "0, 1")
                
    def reducer(self, word, values):
        w0Total=0
        w1Total=0
        for value in values:
            w0, w1 = value.split(",")
            w0Total+=float(w0)
            w1Total+=float(w1)
        self.modelStats[word]= [w0Total, w1Total]
        word, [w0Total, w1Total] 
        
    def reducer_inter(self):
        if self.modelStats[k][0]+self.modelStats[k][1]>3:
            word, self.modelStats[k][0], self.modelStats[k][]
        
    def reducer_final(self):
        class0Total = 1
        class1Total = 1
        for k in self.modelStats.keys():
            if k != "Priori":
                class0Total += self.modelStats[k][0]
                class1Total += self.modelStats[k][1]
        vocabularySize = len(self.modelStats.keys()) -1  #ignore TomsPriors
        #some yields to see some model internal parameters
        #yield ("defaultPrior 0 class", class0Total+vocabularySize)
        #yield ("defaultPrior 1 class", class1Total+vocabularySize)
        #yield ("count 0 class", class0Total)
        #yield ("count 1 class", class1Total)
        #yield ("vocabularySize", vocabularySize)
        
        #calculate priors 
        classCount0, classCount1 = self.modelStats.get("Priori")
        del self.modelStats["Priori"]
        total = classCount0 + classCount1
        yield("Priori", ','.join(str(j) for j in [classCount0, classCount1, classCount0/total, classCount1/total])) 
        for k in self.modelStats.keys():
            yield(k, ','.join(str(j) for j in [self.modelStats[k][0],
                      self.modelStats[k][1],
                      (self.modelStats[k][0] + 1) /(class0Total + vocabularySize), 
                      (self.modelStats[k][1] +1)/(class1Total+vocabularySize)]))  
            
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.reducer),
            MRStep(reducer=self.reducer_inter),
            MRStep(reducer=self.reducer_final)
        ]

if __name__ == '__main__':
    SpamTrainerWithSmoothing_gt3.run()

Overwriting SpamTrainerWithSmoothing_gt3.py


In [74]:
!python SpamTrainerWithSmoothing_gt3.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt 

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.020440.671054
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.020440.671054/output...
Removing temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.020440.671054...


In [93]:
!python SpamTrainerWithSmoothing_gt3.py --jobconf mapred.reduce.tasks=1 HW4/enronemail_1h.txt >HW4/SpamClassWithSmoothing_gt3.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.021812.787247
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
Streaming final output from /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.021812.787247/output...
Removing temp directory /var/folders/3h/lfpv5ykj6414h50t_n_nfv28f2dtvc/T/SpamTrainerWithSmoothing_gt3.z00193k.20160906.021812.787247...


In [94]:
!python SpamClassifier.py --jobconf mapred.reduce.tasks=1 HW4/SpamClassWithSmoothing_gt3.txt

  File "SpamClassifier.py", line 24
SyntaxError: Non-ASCII character '\xe2' in file SpamClassifier.py on line 24, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details
